# Zadatak

Na adresi https://www.foi.unizg.hr/hr/dokumenti nalazi se baza dokumenata Fakulteta organizacije i informatike. Većina dokumenata zadana je u PDF formatu, no neki od dokumenata su čitki (vektorski format) dok su drugi skenirani (rasterski format). Potrebno je:
1) Implementirati program koji će skinuti sve PDF dokumente na lokalno računalo (ili Google Colab / Drive direktorij)
2) Kreirati bazu podataka (SQLite) koja će sadržavati jednu tablicu "dokument". Tablica treba sadržavati šifru dokumenta (autonumber), naslov dokumenta, putanju datoteke (path i filename na lokalnom računalu), URL adresu s koje je dokument skinut, datum, te tekstualni sadržaj dokumenta.
3) Posebno za zadnje polje u tablici potrebno je svaki dokument učitati i ekstrahirati tekst (primjerice putem Python modula PDFMiner - https://github.com/pdfminer/pdfminer.six). Ukoliko je dokument skeniran potrebno je koristiti odgovarajući OCR modul za ekstrakciju teksta (npr. PyTesseract https://pypi.org/project/pytesseract/).
4) Omogućiti pristup skupu podataka putem REST API-ja koji omogućuje isključivo pretraživanje podataka putem:(a) pretraživanja ključnih riječi u tekstu (npr. boolean search), (b) pretraživanje ključnih riječi u naslovu (c) pregled prema datumu (npr. dokumenti od DATUM do DATUM), (d) izlistavanje svih dokumenata.

# Prikupljanje podataka

In [1]:
!mkdir -p data/pdfs data/dummy_data

In [1]:
import os
import sys
import csv
import datetime
import time

import requests
from bs4 import BeautifulSoup
from requests_html import HTMLSession

import numpy as np
import pandas as pd

from sqlalchemy import (create_engine, MetaData, Table, Column, Integer, Text,
                        String, Double, DateTime, insert, select, update, delete)

import fitz
import pytesseract
from PIL import Image
from io import BytesIO
from pdfminer.high_level import extract_text

In [2]:
s = HTMLSession()

def get_data(url, timeout=3, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            response = s.get(url, timeout=timeout)
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        except requests.exceptions.Timeout:
            print(f"Request timed out for URL: {url}. Retrying ({retries + 1}/{max_retries})...")
            retries += 1
        except requests.exceptions.RequestException as e:
            print(f"Error during request to {url}: {e}")
            break  # Break the loop on non-timeout errors

    print(f"Failed to fetch data from {url} after {max_retries} retries.")
    return None

## Downloading PDF

In [3]:
def download_pdf2(url, path, max_retries=3):
    '''
    Download a document from a given URL to a specified directory.
    URL format: /sites/default/files/[document_name].pdf
    '''
    retries = 0
    while retries < max_retries:
        try:
            response = s.get(url)
            response.raise_for_status()  # Checks for HTTP errors

            file_name = url.split('/')[-1]
            file_path = os.path.join(path, file_name)

            with open(file_path, 'wb') as f:
                f.write(response.content)

            print(f"Success: {file_path}")
            break  # Exit loop on successful download
        except requests.exceptions.RequestException as e:
            print(f"Attempt {retries + 1} failed: {e}")
            retries += 1

    if retries == max_retries:
        print(f"Failed to download file from {url} after {max_retries} retries.")

In [4]:
# fixed version
def download_pdf(url, path, max_retries=3, timeout=10):
    '''
    Download a document from a given URL to a specified directory.
    URL format: /sites/default/files/[document_name].pdf
    '''
    retries = 0
    backoff = 1  # Initial backoff duration in seconds

    while retries < max_retries:
        try:
            # Check response headers before downloading the full content
            with s.get(url, stream=True, timeout=timeout) as r:
                r.raise_for_status()  # Checks for HTTP errors

                file_name = url.split('/')[-1]
                file_path = os.path.join(path, file_name)

                with open(file_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:  # Filter out keep-alive chunks
                            f.write(chunk)

            print(f"Success: {file_path}")
            break  # Exit loop on successful download
        except requests.exceptions.RequestException as e:
            print(f"Attempt {retries + 1} failed: {e}")
            sleep(backoff)  # Wait before retrying
            backoff *= 2  # Exponential backoff
            retries += 1

    if retries == max_retries:
        print(f"Failed to download file from {url} after {max_retries} retries.")

## Creating a CSV data entry/extracting useful fields

In [5]:
# fixed version
def get_metadata(url):
    '''
        Dohvaca meta podatke za zapis sa dane url adrese (/hr/dokument/[ime_dokumenta])
    '''
    soup = get_data(url)

    if not soup:
        return (None,) * 5

    ime_datoteke, link1, datum, vrsta_dokumenta, kategorija_dokumenta = (None,) * 5

    try:
        link_element = soup.find('a', href=True, type=lambda value: value and 'application/pdf' in value)
        if link_element:
            link1 = link_element['href']

        datum_element = soup.find('div', class_='datum')
        if datum_element:
            datum_text = datum_element.text.strip()
            datum = datum_text.split('Kreirano: ')[1] if 'Kreirano: ' in datum_text else None

        fields = soup.find_all('div', class_='field-item even')
        if fields:
            ime_datoteke = fields[0].text.strip() if len(fields) > 0 else None
            vrsta_dokumenta = fields[1].text.strip() if len(fields) > 1 else None
            kategorija_dokumenta = fields[2].text.strip() if len(fields) > 2 else None

    except Exception as e:
        print("Error while extracting data:", e)

    return ime_datoteke, link1, datum, vrsta_dokumenta, kategorija_dokumenta

In [6]:
metadata = get_metadata('https://www.foi.unizg.hr/hr/dokument/izmjenjena-odluka-pok-2023')
metadata

('odluka_o_izmjeni_odluke_pok_2023_10_12.pdf',
 'https://www.foi.unizg.hr/sites/default/files/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf',
 '18.12.2023',
 'Odluka',
 'Kvaliteta')

## Handling pagination and going through document on the main pages

In [7]:
def extract_links(span_list):
    return [(span.find('a', href = True, target = False)['href'], span.text) for span in span_list]

In [8]:
soup = get_data('https://www.foi.unizg.hr/hr/dokumenti')
spans = soup.find_all('span', class_ = 'field-content') if soup else None

In [9]:
extract_links(spans)

[('/hr/dokument/izmjenjena-odluka-pok-2023', 'Izmjenjena odluka POK-a 2023.'),
 ('/hr/dokument/odluka-o-raspisivanju-izvanrednih-izbora-studentskog-zbora-suzgfoi',
  'Odluka o raspisivanju izvanrednih izbora Studentskog zbora SUZGFOI'),
 ('/hr/dokument/posebni-dio-financijskog-plana-2024-2026',
  'Posebni dio financijskog plana 2024-2026'),
 ('/hr/dokument/opci-dio-prijedlog-financijskog-plana-2024-2026',
  'Opći dio - prijedlog financijskog plana 2024-2026'),
 ('/hr/dokument/obrazlozenje-posebnog-dijela-financijskog-plana-2024-2026',
  'Obrazloženje posebnog dijela financijskog plana 2024-2026'),
 ('/hr/dokument/obrazlozenje-opceg-dijela-financijskog-plana-2024-2026',
  'Obrazloženje općeg dijela financijskog plana 2024-2026'),
 ('/hr/dokument/dnevni-red-fakultetskog-vijeca-07122023',
  'Dnevni red Fakultetskog vijeća 07.12.2023.'),
 ('/hr/dokument/politika-privatnosti-foi-ja', 'Politika privatnosti FOI-ja'),
 ('/hr/dokument/popis-gospodarskih-subjekata-s-kojima-se-ne-smije-sklapati-u

### Handling pagination

In [10]:
soup = get_data('https://www.foi.unizg.hr/hr/dokumenti')

In [11]:
def get_next_page(soup):
    root_url = 'https://www.foi.unizg.hr'
    page = soup.find('ul', class_ = 'pager')
    next_btn = page.find('li', class_ = 'pager-next')
    if next_btn:
        url = root_url + str(next_btn.find('a')['href'])
        return url
    else:
        return None

In [12]:
def get_urls(url):
    urls = []

    while True:

        soup = get_data(url)

        url = get_next_page(soup)
        if not url:
            break
    
        print(f'Appending {url}')
        urls.append(url)
        
    return urls

In [13]:
urls = get_urls('https://www.foi.unizg.hr/hr/dokumenti')

Appending https://www.foi.unizg.hr/hr/dokumenti?page=1
Appending https://www.foi.unizg.hr/hr/dokumenti?page=2
Appending https://www.foi.unizg.hr/hr/dokumenti?page=3
Appending https://www.foi.unizg.hr/hr/dokumenti?page=4
Appending https://www.foi.unizg.hr/hr/dokumenti?page=5
Appending https://www.foi.unizg.hr/hr/dokumenti?page=6
Appending https://www.foi.unizg.hr/hr/dokumenti?page=7
Appending https://www.foi.unizg.hr/hr/dokumenti?page=8
Appending https://www.foi.unizg.hr/hr/dokumenti?page=9
Appending https://www.foi.unizg.hr/hr/dokumenti?page=10
Appending https://www.foi.unizg.hr/hr/dokumenti?page=11
Appending https://www.foi.unizg.hr/hr/dokumenti?page=12
Appending https://www.foi.unizg.hr/hr/dokumenti?page=13
Appending https://www.foi.unizg.hr/hr/dokumenti?page=14
Appending https://www.foi.unizg.hr/hr/dokumenti?page=15
Appending https://www.foi.unizg.hr/hr/dokumenti?page=16
Appending https://www.foi.unizg.hr/hr/dokumenti?page=17
Appending https://www.foi.unizg.hr/hr/dokumenti?page=18
A

In [14]:
urls[:5]

['https://www.foi.unizg.hr/hr/dokumenti?page=1',
 'https://www.foi.unizg.hr/hr/dokumenti?page=2',
 'https://www.foi.unizg.hr/hr/dokumenti?page=3',
 'https://www.foi.unizg.hr/hr/dokumenti?page=4',
 'https://www.foi.unizg.hr/hr/dokumenti?page=5']

## Downloading in batches

In [15]:
def scrape_page(url):
    '''
        Prolazi kroz linkove glavne stranice, ekstrahira /hr/dokument/ linkove i
        dohvaca meta podatke za svaki dokument
    '''
    root_url = 'https://www.foi.unizg.hr'
    print(f'Scraping {url}')
    
    soup = get_data(url)
    spans = soup.find_all('span', class_ = 'field-content')

    links = extract_links(spans)

    metadata = [get_metadata(root_url + link[0]) for link in links]

    df1 = pd.DataFrame(metadata, columns = ['naziv_datoteke', 'pdf_link', 'datum', 'vrsta_dokumenta', 'kategorija_dokumenta'])
    df2 = pd.DataFrame(links, columns = ['metadata_link', 'naslov_dokumenta'])

    return pd.concat([df1, df2], axis = 1)

In [23]:
df = scrape_page('https://www.foi.unizg.hr/hr/dokumenti')

for url in urls:
    new_df = scrape_page(url)
    df = pd.concat([df, new_df], axis = 0, ignore_index = True)

Scraping https://www.foi.unizg.hr/hr/dokumenti
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=1
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=2
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=3
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=4
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=5
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=6
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=7
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=8
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=9
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=10
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=11
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=12
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=13
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=14
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=15
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=16
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=17
Scraping https://www.foi.un

In [22]:
# Sanity check
df.shape

(933, 7)

In [23]:
# Ciscenje dataframea
df.dropna(subset = ['pdf_link'], inplace = True)

In [24]:
df.shape

(868, 7)

In [25]:
df.head()

,naziv_datoteke,pdf_link,datum,vrsta_dokumenta,kategorija_dokumenta,metadata_link,naslov_dokumenta
0,odluka_o_izmjeni_odluke_pok_2023_10_12.pdf,https://www.foi.unizg.hr/sites/default/files/o...,18.12.2023,Odluka,Kvaliteta,/hr/dokument/izmjenjena-odluka-pok-2023,Izmjenjena odluka POK-a 2023.
1,odluka-izvanredni_izbori_za_studentski_zbor-si...,https://www.foi.unizg.hr/sites/default/files/o...,12.12.2023,Odluka,Studentski zbor,/hr/dokument/odluka-o-raspisivanju-izvanrednih...,Odluka o raspisivanju izvanrednih izbora Stude...
6,fv_dnevni_red_2023-12-07_3._sjednica.pdf,https://www.foi.unizg.hr/sites/default/files/f...,07.12.2023,Dnevni red,Fakultetsko vijeće,/hr/dokument/dnevni-red-fakultetskog-vijeca-07...,Dnevni red Fakultetskog vijeća 07.12.2023.
7,politika-privatnosti-foi.pdf,https://www.foi.unizg.hr/sites/default/files/p...,24.11.2023,None,None,/hr/dokument/politika-privatnosti-foi-ja,Politika privatnosti FOI-ja
8,popis_gs_u_sukobu_interesa_2023.pdf,https://www.foi.unizg.hr/sites/default/files/p...,20.11.2023,Odluka,Javna nabava,/hr/dokument/popis-gospodarskih-subjekata-s-ko...,Popis gospodarskih subjekata s kojima se ne sm...


# SQLAlchemy

In [16]:
# Create an in-memory SQLite database
engine = create_engine('sqlite:///dokumenti.db')

metadata = MetaData()

# Define the "dokument" table with specified columns
dokument = Table('dokument', metadata,
                 Column('sifra_dokumenta', Integer, primary_key=True, autoincrement=True),
                 Column('naslov_dokumenta', String),
                 Column('putanja_datoteke', String),
                 Column('url_adresa', String),
                 Column('datum', DateTime, default=datetime.datetime.utcnow),
                 Column('tekstualni_sadrzaj', Text))

# Create the table
metadata.create_all(engine)

In [17]:
# Checking the created table
dokument.columns.keys()

['sifra_dokumenta',
 'naslov_dokumenta',
 'putanja_datoteke',
 'url_adresa',
 'datum',
 'tekstualni_sadrzaj']

# PDF text extraction

In [24]:
def is_scanned_pdf(file_path):
    text = extract_text(file_path)
    return len(text.strip()) == 0

def extract_text_from_scanned_pdf(file_path):
    document = fitz.open(file_path)
    text = ""

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        pix = page.get_pixmap()
        image = Image.open(BytesIO(pix.tobytes()))
        text += pytesseract.image_to_string(image, lang = 'hrv')

    document.close()
    return text

def extract_text_from_pdf(file_path):
    if is_scanned_pdf(file_path):
        return extract_text_from_scanned_pdf(file_path)
    else:
        return extract_text(file_path)

In [21]:
# Example usage
file_path = './data/dummy_data/scanned.pdf'
is_scanned_pdf(file_path)

True

In [22]:
text = extract_text_from_pdf(file_path)
text

"foi == =\n\nKLASA: 303-02/23-0272\nLURBROJ: 2186-62.01-23-135\nVara, 17. studenog 2023,\n\nTemeljem élanka 60. st. 2. to, 1. Zakona o javnoj nabavi (Narodne novine broj 120/2016, 14/2022),\nFlaite organizacie informatike Varaddin, Pavinska 2, 0: 02024882310 objavjuje\n\nOPIS GOSPODARSKIH SUBJEKATA S KOJIMA NE SMUE SKLAPATI\n‘UGOVORE 0 JAVNOJ NABAVI\n\nUsmis enka 76, Zakona ojanojnabai(Narodnenovine bro 12072018, 1142022) | Oke Fakta\ncrganizaco | informatike Varaddin od dana 20.11.2018. godine, Klasa: 303-028-022, Urbo|: 218682.\n08-1179, suob interesa posi se sledetim gospodarski subjektima\n\nTomisd.o0, E: Kvatemika 15, Varatdin, CIB: 05963033042\n\n(QGP ANALITIKA d.0.0, Draka uica 15, Vrain, O18: 68221757656\n\nAGROPROTEINKA dd, Strojarska cesta 11, Sesvete, OB: 60695452345\n\n'AGROPROTEINKA - ENERGUA do.0, Strojarska cesta 11, 10360 Sesvete, O18: 90174096121\n\nGoolfTech do‘, Ivana Ranger 18, 42000 Varain, OB: 32829800021\n\nMCS doo, Pojska 61, trahoninec, O1: 71383013024\n\n48}, 

In [106]:
df['text'] = None

In [95]:
# download pdf -> extract text -> put it the text field
link = df.iloc[0].pdf_link
download_pdf(link, './data/dummy_data')

Success: ./data/dummy_data/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf


In [96]:
file_path = './data/dummy_data/' + link.split('/')[-1]
file_path

'./data/dummy_data/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf'

In [102]:
text = extract_text_from_pdf(file_path)
df.at[0, 'text'] = text
df.head()

,naziv_datoteke,pdf_link,datum,vrsta_dokumenta,kategorija_dokumenta,metadata_link,naslov_dokumenta,text
0,odluka_o_izmjeni_odluke_pok_2023_10_12.pdf,https://www.foi.unizg.hr/sites/default/files/o...,18.12.2023,Odluka,Kvaliteta,/hr/dokument/izmjenjena-odluka-pok-2023,Izmjenjena odluka POK-a 2023.,KLASA: 602-04/23-06/1 \nURBROJ: 2186-62-06-23-...
1,odluka-izvanredni_izbori_za_studentski_zbor-si...,https://www.foi.unizg.hr/sites/default/files/o...,12.12.2023,Odluka,Studentski zbor,/hr/dokument/odluka-o-raspisivanju-izvanrednih...,Odluka o raspisivanju izvanrednih izbora Stude...,None
6,fv_dnevni_red_2023-12-07_3._sjednica.pdf,https://www.foi.unizg.hr/sites/default/files/f...,07.12.2023,Dnevni red,Fakultetsko vijeće,/hr/dokument/dnevni-red-fakultetskog-vijeca-07...,Dnevni red Fakultetskog vijeća 07.12.2023.,None
7,politika-privatnosti-foi.pdf,https://www.foi.unizg.hr/sites/default/files/p...,24.11.2023,None,None,/hr/dokument/politika-privatnosti-foi-ja,Politika privatnosti FOI-ja,None
8,popis_gs_u_sukobu_interesa_2023.pdf,https://www.foi.unizg.hr/sites/default/files/p...,20.11.2023,Odluka,Javna nabava,/hr/dokument/popis-gospodarskih-subjekata-s-ko...,Popis gospodarskih subjekata s kojima se ne sm...,None


In [109]:
text.strip()

'KLASA: 602-04/23-06/1 \nURBROJ: 2186-62-06-23-76 \nVaraždin, 19. listopada 2023. \n\nNa temelju članka 53. Statuta Sveučilišta u Zagrebu Fakulteta organizacije i informatike (dalje u \ntekstu: Fakultet) i članka 8. i 9. Pravilnika o sustavu osiguravanja kvalitete Sveučilišta u Zagrebu \nFakulteta organizacije i informatike (u daljnjem tekstu: Pravilnik), Fakultetsko vijeće Fakulteta, \nna sjednici održanoj 19. listopada 2023. godine, donosi \n\nO D L U K U  \n\nI. \nProf. dr. sc. Renata Mekovec imenuje se za predsjednicu Povjerenstva za osiguravanje kvalitete \nSveučilišta  u  Zagrebu  Fakulteta  organizacije  i  informatike  (u  daljnjem  tekstu:  Povjerenstvo) \numjesto dosadašnjeg predsjednika prof. dr. sc. Zlatka Erjavca, a koji će kao član Uprave Fakulteta \ndjelovati u Povjerenstvu kao predstavnik Uprave i koordinator Povjerenstva.  \n\nProf.  dr.  sc.  Sandra  Lovrenčić  imenuje  se  za  zamjenicu  predsjednice  Povjerenstva  umjesto \ndosadašnje zamjenice predsjednika Povjeren